In [ ]:
!pip install bi-lstm-crf
! wget https://github.com/kobkrit/datasets/raw/main/AIFORTHAI-LST20Corpus.tar.gz
! tar -xvzf AIFORTHAI-LST20Corpus.tar.gz

In [ ]:
!pip install datasets -qq

In [ ]:
#cell 0
from datasets import load_dataset
import pandas as pd
from torch.utils.data import DataLoader,Dataset

import torch as T
import torch.nn as N
import torch.optim as O

import random
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import os

In [ ]:
lst20 = load_dataset("lst20", data_dir="LST20_Corpus")
lst20

In [ ]:
# len(os.listdir("/content/LST20_Corpus/test"))

In [ ]:
data_df = pd.DataFrame(lst20['train'])[['tokens']]
data_df

In [ ]:
data_val = pd.DataFrame(lst20['validation'])
data_test = pd.DataFrame(lst20['test'])

In [ ]:
data_df['tokens'] = data_df.tokens.apply(lambda x:'|'.join(x).replace('_',' ').split('|'))
data_df['len'] = data_df.tokens.apply(len)
data_df['lenchar'] = data_df.tokens.apply(lambda x:len(''.join(x)))
data_df

In [ ]:
data_df = data_df[data_df.len > 10].reset_index()
data_df

In [ ]:
#cell 7
"""char2idx = {None: 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, '+': 12, ',': 13, '-': 14, '.': 15, '/': 16, '0': 17, '1': 18, '2': 19, '3': 20, '4': 21, '5': 22, '6': 23, '7': 24, '8': 25, '9': 26, ':': 27, ';': 28, '=': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '[': 58, ']': 59, 'a': 60, 'b': 61, 'c': 62, 'd': 63, 'e': 64, 'f': 65, 'g': 66, 'h': 67, 'i': 68, 'j': 69, 'k': 70, 'l': 71, 'm': 72, 'n': 73, 'o': 74, 'p': 75, 'q': 76, 'r': 77, 's': 78, 't': 79, 'u': 80, 'v': 81, 'w': 82, 'x': 83, 'y': 84, 'z': 85, '\xa0': 86, '®': 87, 'é': 88, 'ü': 89, 'ก': 90, 'ข': 91, 'ฃ': 92, 'ค': 93, 'ฅ': 94, 'ฆ': 95, 'ง': 96, 'จ': 97, 'ฉ': 98, 'ช': 99, 'ซ': 100, 'ฌ': 101, 'ญ': 102, 'ฎ': 103, 'ฏ': 104, 'ฐ': 105, 'ฑ': 106, 'ฒ': 107, 'ณ': 108, 'ด': 109, 'ต': 110, 'ถ': 111, 'ท': 112, 'ธ': 113, 'น': 114, 'บ': 115, 'ป': 116, 'ผ': 117, 'ฝ': 118, 'พ': 119, 'ฟ': 120, 'ภ': 121, 'ม': 122, 'ย': 123, 'ร': 124, 'ฤ': 125, 'ล': 126, 'ฦ': 127, 'ว': 128, 'ศ': 129, 'ษ': 130, 'ส': 131, 'ห': 132, 'ฬ': 133, 'อ': 134, 'ฮ': 135, 'ฯ': 136, 'ะ': 137, 'ั': 138, 'า': 139, 'ำ': 140, 'ิ': 141, 'ี': 142, 'ึ': 143, 'ื': 144, 'ุ': 145, 'ู': 146, 'ฺ': 147, '฿': 148, 'เ': 149, 'แ': 150, 'โ': 151, 'ใ': 152, 'ไ': 153, 'ๅ': 154, 'ๆ': 155, '็': 156, '่': 157, '้': 158, '๊': 159, '๋': 160, '์': 161, 'ํ': 162, '๎': 163, '๏': 164, '๐': 165, '๑': 166, '๒': 167, '๓': 168, '๔': 169, '๕': 170, '๖': 171, '๗': 172, '๘': 173, '๙': 174, '๚': 175, '๛': 176, '\u200e': 177, '–': 178, '—': 179, '‘': 180, '’': 181, '…': 182, '™': 183}
"""

#cell 8
idx2char = [' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '®', 'é', 'ü', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ', 'ล', 'ฦ', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า', 'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', '฿', 'เ', 'แ', 'โ', 'ใ', 'ไ', 'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๎', '๏', '๐', '๑', '๒', '๓', '๔', '๕', '๖', '๗', '๘', '๙', '๚', '๛', '\u200e', '–', '—', '‘', '’', '…', '™']

char2idx={v:k for k,v in enumerate(idx2char)}


In [ ]:
#cell 9
def str2idxseq(charseq):
    idxseq = []
    for char in charseq:
        char = char.lower()
        idxseq.append(char2idx.get(char,0))
        """if char in char2idx:
            idxseq.append(char2idx[char])
        else:
            idxseq.append(char2idx[None])"""
    return idxseq
cuter_idxseq = str2idxseq('สวัสดีครับ')
print(cuter_idxseq)

In [ ]:
def idxseq2str(idxseq):
    charseq = []
    for idx in idxseq:
        if idx < len(idx2char):
            charseq.append(idx2char[idx])
        else:
            charseq.append(' ')
    return charseq
print(''.join(idxseq2str(cuter_idxseq)))

In [ ]:
def sent2data(sent):
    charidxs = []
    wordbrks = []
    for charseq in sent:
        idxs = str2idxseq(charseq)
        charidxs.extend(idxs)
        wordbrks.extend((len(idxs) - 1) * [False] + [True])
    return (T.tensor(charidxs).cuda(), T.tensor(wordbrks).cuda())

In [ ]:

def corpus2dataset(corpus):
    dataset = []
    for sent in corpus:
        charidxs, wordbrks = sent2data(sent)
        dataset.append((charidxs, wordbrks))
    return dataset

In [ ]:
#cell 13
def corpus2dataset_dl(corpus):
    dataset = []
    data = []
    label = []
    for sent in corpus:
        charidxs, wordbrks = sent2data(sent)
        data.append(charidxs)
        label.append(wordbrks)
    return data , label

In [ ]:
train_train , label_train = corpus2dataset_dl(data_df['tokens'].to_list())
train_val , label_val = corpus2dataset_dl(data_val['tokens'].to_list())
train_test , label_test = corpus2dataset_dl(data_test['tokens'].to_list())

In [ ]:
class Mydataset(Dataset):
    def __init__(self, train, label):
        self.train = train
        self.label = label

    def __len__(self):
        return len(self.train)

    def __getitem__(self, idx):
        return self.train[idx],self.label[idx]

In [ ]:
def colletefn(bat):
    a=[]
    b=[]
    for i in bat:
        a.append(i[0])
        b.append(i[1])
    a = T.cat(a)
    b = T.cat(b)
    return a,b


In [ ]:
dataset_train = Mydataset(train=train_train,label=label_train) 
dataset_val = Mydataset(train=train_val,label=label_val) 
dataset_test = Mydataset(train=train_test,label=label_test) 

In [ ]:
train_loader = DataLoader(dataset_train, batch_size=256, shuffle=True, collate_fn=colletefn)
val_loader = DataLoader(dataset_val,batch_size=256, shuffle=True, collate_fn=colletefn)
test_loader = DataLoader(dataset_test, batch_size=256 ,shuffle=True, collate_fn=colletefn)

In [ ]:
def wordbrks2brkvec(wordbrks):
    return wordbrks.bool().long()
    brkvec = T.LongTensor(len(wordbrks))
    for i in range(len(wordbrks)):
        if wordbrks[i]: brkvec[i] = 0
        else: brkvec[i] = 1
    return brkvec.cuda()

#cell 19

In [ ]:
T.cuda.is_available()

In [ ]:
class WordsegModel(N.Module):
    def __init__(self, dim_charvec, dim_trans, no_layers):
        super(WordsegModel, self).__init__()
        self._dim_charvec = dim_charvec
        self._dim_trans = dim_trans
        self._no_layers = no_layers
        
        self._charemb = N.Embedding(184, self._dim_charvec)
        
        self._rnn = N.GRU(
            self._dim_charvec, self._dim_trans, self._no_layers,
            batch_first=True, bidirectional=True ,dropout=0.2
        )

        self._tanh = N.Tanh()
        self._hidden = N.Linear(2 * self._dim_trans, 2)    # Predicting two classes: break / no break
        self._log_softmax = N.LogSoftmax(dim=1)
        
    def forward(self, charidxs):
        # try:
            charvecs = self._charemb(T.as_tensor(charidxs,device='cuda'))
            # print('charvecs =\n{}'.format(charvecs))
            ctxvecs, lasthids = self._rnn(charvecs.unsqueeze(0))
            ctxvecs, lasthids = ctxvecs.squeeze(0), lasthids.squeeze(1)
            # print('ctxvecs =\n{}'.format(ctxvecs))
            statevecs = self._hidden(self._tanh(ctxvecs))
            # print('statevecs =\n{}'.format(statevecs))
            brkvecs = self._log_softmax(statevecs)
            # print('brkvecs =\n{}'.format(brkvecs))
            return brkvecs
        # except RuntimeError:
        #     raise RuntimeError(statevecs)

#cell 21

In [ ]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = T.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return T.tensor(idxs, dtype=T.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        T.log(T.sum(T.exp(vec - max_score_broadcast)))

In [ ]:
class BiLSTM_CRF(N.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = N.Embedding(vocab_size, embedding_dim)
        self.lstm = N.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = N.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = N.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:

device = T.device("cuda" if T.cuda.is_available() else "cpu")
device

In [ ]:
# !pip install gdown
# !gdown --id 10ehc_ARUyzkWycJekS51_bIdzU3exBSQ

In [ ]:
model = WordsegModel(32,256,4).to(device=device)
model

In [ ]:
def train_model(wordseg_model, training_data, val_data,epochs, loss_fn, optimizer):
    no_samples = len(training_data)
    loss_history = []
    no_val = len(val_data)
    val_loss_his = []
    
    for i in range(epochs):
        total_loss = 0.0
        # random.shuffle(training_data)
        v_loss = 0.0
        # random.shuffle(val_data)
        for charidxs, wordbrks in tqdm(training_data):   
            # charidxs = T.flatten(charidxs)
            # wordbrks = T.flatten(wordbrks)
            pred_brkvecs = wordseg_model(charidxs)       # Perform prediction
            gold_brkvec = wordbrks2brkvec(wordbrks)      # Gold standard
            pred_brkvecs = pred_brkvecs.cuda()
            # print(gold_brkvec)

            loss = loss_fn(pred_brkvecs, gold_brkvec)
            loss.cuda()
            total_loss += loss.item()
            
            optimizer.zero_grad()      # Clear gradient cache
            loss.backward()            # Perform backpropagation
            optimizer.step()           # Update the model parameters
        for (charidxs, wordbrks) in tqdm(val_data):   
        
            pred_brkvecs = wordseg_model(charidxs)       # Perform prediction

            gold_brkvec = wordbrks2brkvec(wordbrks)      # Gold standard

            lossv = loss_fn(pred_brkvecs, gold_brkvec)
            v_loss += lossv.item()

        
        # wandb.log({"loss": total_loss / no_samples})
        print("epochs {}   Loss: {}  val_loss: {}".format(i+1,(total_loss / no_samples),(v_loss/no_samples)))
        # sentline("epochs {}\nLoss: {}  \nval_loss: {}".format(i+1,(total_loss / no_samples),(v_loss/no_samples)))
        loss_history.append(total_loss / no_samples)
        val_loss_his.append(v_loss / no_val)
        T.save(wordseg_model.state_dict(), 'best_{}.pt'.format(i+1))
    # epoch_count = range(1, epochs + 1)
    # plt.plot(epoch_count, loss_history, 'b--')
    # plt.plot(epoch_count, val_loss_his, 'r')
    # plt.legend(['Training Loss'])
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.show()

    return loss_history



In [ ]:
#cell 24
learning_rate = 0.01
epochs = 20

# wordseg_model = wordseg_model.cuda()
loss_fn = N.NLLLoss()
optimizer = O.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

loss_history = train_model(model, train_loader, val_loader,epochs, loss_fn, optimizer)

T.save(model.state_dict(), 'best_final.pt')

#cell 25